Each team will develop a machine learning model using those available in `scikit-learn` to predict whether or not a particular will elect to stop services provided by a financial institution based on known attributes. The first five rows of the training data are shown below.

In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
train.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,597,Germany,Female,35,8,131101.04,1,1,1,192852.67,0
1,523,France,Female,40,2,102967.41,1,1,0,128702.10,1
2,706,Spain,Female,42,8,95386.82,1,1,1,75732.25,0
3,788,France,Male,32,4,112079.58,1,0,0,89368.59,0
4,706,Germany,Male,38,5,163034.82,2,1,1,135662.17,0


Each team will deploy there models to Azure. Specifically, each team should use `Flask` to deploy a web service that can accept JSON payloads via a `post` request. An example of a typical JSON payload that should be expected is shown below.

In [3]:
target = 'Exited'
features = [col for col in train.columns if col != target]

train[features].loc[0].to_dict()

{'CreditScore': 597,
 'Geography': 'Germany',
 'Gender': 'Female',
 'Age': 35,
 'Tenure': 8,
 'Balance': 131101.04,
 'NumOfProducts': 1,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 192852.67}

**This will be a graded assignment!** Points will be allocated as follows:
- 70 points for a working deployment
- 20 points for a model that achieves a predictive accuracy greater than 80%.
- 10 points based on competition

The model competition will take place on Monday, 2/21. Each model will be asked to make predictions for > 30 unseen customers. The winning model will be the one that achieves the **highest predictive accuracy**. Teams with the **best and worst** performing models will give a overview of their modeling pipeline (all teams should be prepared to speak to this). This overview should include discussions on:
- data preparation steps,
- model selection, and
- model tuning.

In [5]:
train.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [7]:
train.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64